In [1]:
# from eval import liHongEvaluator
from sklearn.neural_network import MLPRegressor
import random
import tensorflow as tf
# from neural_model import TFNeuralNetworkRegressor
# from get_onehotencoded_structure import get_structure1,dict_arm_to_structure1
import sys
import os
import numpy as np
from datetime import datetime
from math import log
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import random
import json
from keras.layers import LSTM
# from conversions import get_structure1,dict_arm_to_structure1
from sklearn.model_selection import train_test_split
import os
# from getData import campaign_def
import keras
#from getData import campaign_def
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import RepeatVector
from tflearn import DNN
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression

import pickle

/home/anubha/anaconda3/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anubha/anaconda3/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anubha/anaconda3/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anubha/anaconda3/envs/venv/lib/python3.6/site-

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
tf.reset_default_graph()

In [3]:
class liHongEvaluator():

    def __init__(self, main_folder, output_column_name, print_params = {}, use_non_stationarity = False, sampling_fraction=0.1, stochastic_param = [1,0]):
        self.main_folder = main_folder
        self.df_original = pd.read_csv(main_folder,engine='python')
#         self.df_original=self.df_original.drop(columns=['user_id','article_id'])
        print(self.df_original.head())

        all_columns = list(self.df_original.columns)
        self.output_column_names = ['words_count',
       'category_26', 'category_66', 'category_136', 'category_186',
       'category_209', 'category_247', 'category_250', 'category_265',
       'category_281', 'category_301', 'category_327', 'category_331',
       'category_348', 'category_351', 'category_354', 'category_375',
       'category_409', 'category_412', 'category_418', 'category_431',
       'category_437','29953','39894','70591','87224','95972','96663','119592',
       '123290','123818','123826','124176','140528','156625','158082','160474',
       '160940','161425','182394','202557','206934','214800','215861','225019','225055',
                                    '235132','236207','236338','236444','236951','276946',
                                    '283392','283505','284096','284583','285343','285377',
                                    '288435','313920','337143','338339','created_at_ts']

         

        self.input_columns = list(set(all_columns) - set(self.output_column_names))
        
#         print(self.input_columns)
        self.arm_column_names = self.output_column_names
        self.logging_frequency = 1000
        self.number_features_in_context = len(self.input_columns)
        self.no_oflist_arms = len(self.output_column_names)
        total_arms =40

        #For testing, Comment out
        self.count_not_found = 0
        self.found_in_exclusion = 0
        self.found_in_set = 0
        self.flag = True
        x = self.df_original[self.input_columns].values
        x_scaled=pd.DataFrame(x,columns = self.input_columns)
        self.unique = self.df_original[self.output_column_names].drop_duplicates()
        self.unique = self.unique.reset_index(drop=True)

        self.y=self.df_original[self.output_column_names].values
        #To simulate stochasticity set this paramater
        self.stochastic_param = stochastic_param
        x_tr, x_test , y_tr, y_test = train_test_split(x_scaled,self.y,test_size=0.3,shuffle=False)
        x_train ,x_valid,y_train, y_valid = train_test_split(x_tr,y_tr,test_size=0.3,shuffle=False)
 
        self.x_training = x_train.values.tolist() #list(map(float,x_train.values))#pd.DataFrame(x_tr, columns = self.input_columns)
        self.y_training = y_train #pd.DataFrame(y_tr, columns = [self.output_column_names])
        self.x_valid=x_valid.values.tolist()
        self.y_valid=y_valid
        self.x_testing =x_test.values.tolist()#pd.DataFrame(x_test, columns = self.input_columns)
        self.y_testing = y_test# pd.DataFrame(y_test, columns = [self.output_column_names])
        x_train_df = pd.DataFrame(x_tr, columns = self.input_columns)
        x_test_df = pd.DataFrame(x_test, columns = self.input_columns)
        
        self.e=0.2
        self.decay=0.998
        
        print ('x_traing',self.x_training[0])
        print ('x_valid',self.x_valid[0])
        print ('x_test' ,self.x_testing[0])


        random.seed(9001)

        self.non_stationarity_steps = 25000
        
    def get_size_input(self):
        return self.df.shape[0]

    def get_no_oflist_arms(self):
        return self.no_oflist_arms

    def get_dataset_shape(self):
        return self.df.shape

    def get_number_features_in_context(self):
        return self.number_features_in_context

    #rename to get_one_hot_structure_array
    def get_structure(self):
#         print(np.unique(self.y))
        structure=self.unique
        #print 'structure', structure
        return structure
    #rename to get_one_hot_structure
    def dict_arm_to_structure(self):
        dict_str ={}
        y_unique = self.get_structure()
        print (y_unique.head())
        for i ,r in y_unique.iterrows():
            dict_str[i] = r.values.tolist()
        return dict_str

    def get_exclusions(self):
        arms_set_dict ,self.exclusion_dict = campaign_def(self.main_folder)
        return self.exclusion_dict


    def evaluate(self,policy):
        
        dict_arm_structure = self.dict_arm_to_structure()
        score=0
        total_arms=40
        start_time = datetime.now()
        q=0
        res_rew=[]
        rew_est=0
        

        
        print("now training model")
        for state, right_arm in zip(self.x_training,self.y_training):
            
            q+=1
            if(q%100 == 0):
                res_rew.append(rew_est)
                rew_est=0
            
            
            import random
            rand = random.uniform(0, 1)

            if(rand < self.e):
                self.e=self.e*self.decay
                selected_action =[]
                
                while len(selected_action) <1:
                    temp = self.unique.iloc[np.random.randint(0,total_arms)].values.tolist()
                    if(temp not in selected_action):
                        selected_action.append(temp)
            else:        
            
                selected_action = policy.get_action(state)
                
            reward=0

            for arm in selected_action :
                reward=0
                if arm == right_arm.tolist():

                    reward = 1
                    rew_est+=1

                state2 =list(state) + (arm)
                policy.update_model(reward,state2,arm,True)

            
            #print("!!!", right_arm)
#             state1 = list(state) + list(right_arm)
#             policy.update_model(1,state1,right_arm,True)
            
#             import random
#             random_sample = list(self.unique.iloc[random.randint(0,39)])
#             if random_sample!=list(right_arm):
#                 policy.update_model(0,state + random_sample,random_sample, True)

#             pairs = list(zip(self.x_training,self.y_training))  # make pairs out of the two lists
#             pairs = random.sample(pairs, 1)  # pick 3 random pairs
#             A1, B1 = zip(*pairs)  # separate the pairs
#             for k in range(1):
#                 if(list(B1[k])!=list(right_arm)):
#                     state1 = list(A1[k]) + list(B1[k])
#                     policy.update_model(0,state1,B1[k],True)
                    
            
#             for index, row in self.unique.iterrows():
#                 row = row.to_numpy()
#                 state2 =list( state) + list(row)
#                 #print(state)
#                 if(list(row)==list(right_arm)):
#                     policy.update_model(1,state2,right_arm,True)
#                 else:
#                     policy.update_model(0,state2,row,True)
            
# # # #             print('state',state)
            
# # #         policy.train_model(self.x_training, self.y_training)
        print("model trained")
        
        for state, right_arm in zip(self.x_valid, self.y_valid):
            
            q+=1
            if(q%100 == 0):
                res_rew.append(rew_est)
                rew_est=0

            import random
            rand = random.uniform(0, 1)

            if(rand < self.e):
                self.e=self.e*self.decay
                selected_action =[]

                while len(selected_action) <1:
                    temp = self.unique.iloc[np.random.randint(0,total_arms)].values.tolist()
                    if(temp not in selected_action):
                        selected_action.append(temp)
            else:
                selected_action = policy.get_action(state)
                
            reward=0


            for arm in selected_action :
                reward=0
                if arm == right_arm.tolist():

                    reward = 1
                    rew_est+=1


                state2 =list(state) + (arm)
                policy.update_model(reward,state2,arm,True)

        
        
##############################################TESTING##############################################
        
        
        count =0
        
        precision=0
        dict_recall= {}
        dict_impressions = {}
        dict_clicks = {}
        
        for k in self.unique.values:
            dict_impressions[tuple(k)] =0
            dict_clicks[tuple(k)]=0
        q=0
        ndgc=0
        rew_est =0
        res_rew=[]
        for state, right_arm in zip(self.x_testing, self.y_testing):
                                    
            import random
            rand = random.uniform(0, 1)

            if(rand < self.e):
                self.e=self.e*self.decay
                selected_action =[]

                while len(selected_action) <1:
                    temp = self.unique.iloc[np.random.randint(0,total_arms)].values.tolist()
                    if(temp not in selected_action):
                        selected_action.append(temp)
            else:
                selected_action = policy.get_action(state)
                
                
            q+=1
            if(q%100 == 0):
                res_rew.append(rew_est)
                rew_est=0
            


            y_value = right_arm
    
            

            count=count+1
            self.stime = datetime.now()
            if count % self.logging_frequency == 0 :
                print('evaluated %d' % count)
# #             print('initial', len(state))
            selected_action = policy.get_action(state)
#             print("")
            match_found = False
            ps =0
            ps_sum=0
            i=0
            num_ndgc=0
            d_ndgc=0
            k_ndgc=0
            for arm in selected_action : 
                    reward=0
                    dict_impressions[tuple(arm)]+=1

                    if arm == right_arm.tolist():
                        rew_est +=1

                        match_found = True
                        reward = 1
                        score = score+1

                        ps+=1
                        ps_sum += ps/(i+1)
                        i=i+1
                        
                        if(i+1==1):
                            d_ndgc+=1
                        else:
                            d_ndgc+=1/(log(i+1))
                        num_ndgc = num_ndgc+1
                        
                        dict_clicks[tuple(y_value)] +=1


                    state2 =list(state) + (arm)
                    policy.update_model(reward,state2,arm,True)
                    
            if(match_found):
                precision =precision + ps_sum
                
            for i in range(num_ndgc):
                if(i+1 == 1):
                    k_ndgc+=1
                else:
                    k_ndgc+= 1/(log(i+1))
            
            if(k_ndgc!=0):
                ndgc = ndgc + (d_ndgc/k_ndgc)
            


        print('accuracy %f' % ( float(score) / count ))
        
        ctr = 0 
        for key, value in dict_impressions.items():
            if(value!=0):
                ctr += dict_clicks[key]/value
        print('ctr',ctr/len(dict_impressions.keys()))
        
        with open('result_reward_single_news_ts1.pickle', 'wb') as handle:
            pickle.dump(res_rew, handle)
        
        print ("precision")
        print (precision/len(self.x_testing))
        
                
        print ("NDGC")
        print (ndgc/len(self.x_testing))
        




In [4]:
pwd

'/media/anubha/Seagate Expansion Drive/china'

In [5]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
data_path = '/media/anubha/Seagate Expansion Drive/china'
class TFNeuralNetworkRegressor():
    
    


    def __init__(self, number_features_in_context, number_features_in_structure):
        
        
        self.pre_model = Sequential()
        self.pre_model.add(LSTM(32, batch_input_shape=(1,1,number_features_in_context+number_features_in_structure),stateful=True,name='layer1'))
        self.pre_model.add(RepeatVector(1))
        self.pre_model.add(LSTM((number_features_in_context+number_features_in_structure), return_sequences=True)) 
        self.pre_model.add(Dense((number_features_in_context+number_features_in_structure),activation ='softmax'))
        self.pre_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
#         checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=0)
        print(self.pre_model.summary())
    
        self.model = Sequential()
        #First Hidden Layer
#         self.model.add(LSTM(120, batch_input_shape=(100,number_features_in_context+number_features_in_structure,1), stateful=True))

        self.model.add(Dense(120, activation='tanh', kernel_initializer='random_normal', input_shape=(32,)))
        
        
        self.model.add(Dense(80, activation='tanh', kernel_initializer='random_normal'))
        
        self.model.add(Dense(40, activation='tanh', kernel_initializer='random_normal'))
        

        self.model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
        # For a binary classification problem
        
        self.model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
        print(self.model.summary())
    
    
        layer_name = 'layer1'
        self.intermediate_layer_model = Model(inputs=self.pre_model.input,outputs=self.pre_model.get_layer(layer_name).output)
    
       
    def fit(self, X, Y):
#         print(X.shape)
        X = X.reshape(X.shape[0],1,X.shape[1])
#         intermediate_output = self.intermediate_layer_model.predict(data)
        self.pre_model.fit(X,X,batch_size=1,verbose=0)
        intermediate_output = self.intermediate_layer_model.predict(X,batch_size=1)
        
#         intermediate_from_pre_model = self.pre_model.get_layer('layer1').output
        
#         print(intermediate_from_pre_model)
#         inp_s = len(intermediate_from_pre_model[0])
#         inter  = intermediate_from_pre_model.tolist().reshape(X.shape[0], X.shape[1])
        self.model.fit(intermediate_output, Y, epochs=5, verbose=0)
        
        
   
    def partial_fit(self, X,Y):
        
        
        self.model.fit(X, Y, epochs=1, verbose=0)
   
    def predict(self, X):
#         print('shape', X.shape)
        X = X.reshape(X.shape[0], 1,X.shape[1])
        intermediate_output = self.intermediate_layer_model.predict(X,batch_size=1)
        
#         intermediate_from_pre_model = self.pre_model.get_layer('lstm_22').output
        
        return self.model.predict(intermediate_output)

In [6]:
def dict_arm_to_structure1():
    with open('total_comb_structured_dict') as f:
        data = json.load(f)
        data1={}
        for i in data.keys():
            j=str(i)
            data1[j]=data[i]
        return data1

def get_structure1():
	total_comb_structured_dict=dict_arm_to_structure1()
	total_structure_array= np.asarray(list(total_comb_structured_dict.values()))
	return total_structure_array

def num_to_key():
    with open('dict_structure_to_arm') as f:
        data = json.load(f)
        data1={}
        for i in data.keys():

            j=str(i)
            data1[j]=data[i]
        return data1


In [7]:
#150 -50-0.7 [150 - 100-50]
class single_model_policy():
    #Taking batch sizes of 1000
    def __init__(self, number_features_in_context, structure, epsilon, training_dropout, prediction_dropout,batch_size = 100):
        self.last_action = None
        self.last_reward = None
        self.last_state = None
        self.num_arms = structure.shape[0]
        self.structure_features = structure.shape[1]
        self.epsilon = epsilon
        self.experience = {}
        self.i=0
        self.total_features = number_features_in_context + self.structure_features
        self.train_batch_size=80
        self.mini_batch=500
        dict_str ={}
        self.cnt=0
      
        for i ,r in structure.iterrows():
            dict_str[i] = r.values.tolist()
            
        
     
    
        self.structure = dict_str
#         print (self.structure)
#         self.batch_size = batch_size
        self.last_s=[]
        self.last_r=[]
        #if self.batch_size:
        self.iter = 0
        self.dict_update_hist = {}
#         print(self.structure)
        # self.model=TFNeuralNetworkRegressor(self.total_features, training_dropout, prediction_dropout)
        self.model=TFNeuralNetworkRegressor(number_features_in_context, self.structure_features)
        


    #Choose best action for given context
    def get_action(self, context):
        #action should be category ID
        best_action = self.get_best_arm(context)
        arms=[]
        for i in best_action:
            arms.append(self.structure[i])
            
        return arms

    #Loop over the structures of every arm for a given context and return the best prediction
    def get_best_arm(self,context):

        best_score = 0
        best_arm = None

        inputs = []
        arms = []
        for arm,structure in self.structure.items():
#             print((structure))
#             print (list())
            state = list(context) + list(structure)
#             print(len(state))
            inputs.append(state)
            arms.append(arm)

        scores = [ elt[0] for elt in list(self.model.predict(np.array(inputs))) ]
#         print(scores)
        n=0
        select_arm=[]
        s = sorted(range(len(scores)),reverse = True,  key=lambda k: scores[k])

        return s[:1]

    
    def train_model(self, x_train, y_train):
        self.model.fit(x_train,y_train)
    #Update model

    def update_model(self, reward,last_state=[] ,last_action=[],update_in_train=False):
        #Immediate Updates
        if(len(last_action) != 0):
            self.last_action = last_action
        if(len(last_state) !=0):
            self.last_state = last_state
        self.last_reward = reward
        self.model.fit(np.array([self.last_state]), np.array([[self.last_reward]]))
#         print (batch_size)
        if update_in_train:
#             if(len(self.last_s)!=80):
            if(self.iter == 0 or self.iter%self.train_batch_size != 0):
#                 if(reward==0):
#                     self.cnt+=1

#                 if(self.cnt<int(0.5*self.train_batch_size) or reward==1):

#                     if(self.last_state not in self.last_s):
                self.last_s.append(self.last_state)
                self.last_r.append(self.last_reward)

            else:

                for iteration in range(1,50):
                    if(iteration%500==0):
                        print("Iteration : %d" % iteration)
                    num_to_sample = min(len(self.last_s), self.mini_batch)
                    random_indices=random.sample(range(0,len(self.last_s)),num_to_sample)
                    x_train = []
                    y_train = []
                    for index in random_indices:

                        x_train.append(self.last_s[index])
                        y_train.append([self.last_r[index]])

                    self.model.fit(np.array(x_train),np.array(y_train))
                self.last_s=[]
                self.last_r=[]
                self.cnt=0
            self.iter+=1


In [8]:
training_dropout, prediction_dropout =  0.5, 0.5
epsilon = 0.00001

main_folder = 'news_context_&_structure_TS_sorted.csv'

target_evaluator = liHongEvaluator(main_folder, ['article_id'] , {}, False, 0.1, [1,0])

print("Created Evaluator")


   session_start  session_size  click_timestamp  click_region  created_at_ts  \
0       0.000986      0.000000         0.000000      0.888889        0.00271   
1       0.000714      0.008197         0.000010      0.444444        0.00000   
2       0.000933      0.016393         0.000024      1.000000        0.00000   
3       0.000862      0.000000         0.000028      0.296296        0.00271   
4       0.000351      0.000000         0.000035      0.740741        0.00271   

   words_count  category_26  category_66  category_136  category_186  ...  \
0     0.543590            0            0             0             0  ...   
1     0.712821            0            0             0             0  ...   
2     0.712821            0            0             0             0  ...   
3     0.543590            0            0             0             0  ...   
4     0.543590            0            0             0             0  ...   

   283392  283505  284096  284583  285343  285377  28843

In [9]:
df=pd.read_csv(main_folder)

In [10]:
structure= target_evaluator.get_structure()
number_features_in_context = target_evaluator.get_number_features_in_context()

In [11]:
smp = single_model_policy(number_features_in_context, structure, epsilon, training_dropout, prediction_dropout)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (LSTM)                (1, 32)                   17152     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (1, 1, 32)                0         
_________________________________________________________________
lstm_1 (LSTM)                (1, 1, 101)               54136     
_________________________________________________________________
dense_1 (Dense)              (1, 1, 101)               10302     
Total params: 81,590
Trainable params: 81,590
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 120)               3960      
_________________________________________________________________
d

In [12]:
target_evaluator.evaluate(smp)


   words_count  category_26  category_66  category_136  category_186  \
0     0.543590            0            0             0             0   
1     0.712821            0            0             0             0   
2     0.866667            0            0             0             0   
3     0.282051            0            0             0             0   
4     1.000000            0            0             0             0   

   category_209  category_247  category_250  category_265  category_281  ...  \
0             1             0             0             0             0  ...   
1             0             1             0             0             0  ...   
2             0             0             0             0             1  ...   
3             0             0             0             0             0  ...   
4             0             0             0             0             0  ...   

   283505  284096  284583  285343  285377  288435  313920  337143  338339  \
0       0

KeyboardInterrupt: 

In [ ]:
batch size 1000
iterations :10 
    _____________
accuracy 0.250963
ctr 0.04623804173875244
precision
0.25096348791758755
NDGC
0.362063779462863

In [ ]:
batch size : 500
iterations : 50
    
    ____________________
    
accuracy 0.410747
ctr 0.14885359288583871
precision
0.4107472282598769
NDGC
0.5925829892693465

In [ ]:
accuracy 0.583919
ctr 0.20346427770087075
precision
0.5839193409925295
NDGC
0.8424175375289795

3 layer
batch size=100
iterations :50

In [ ]:

3 layer 120 -80-40
batch size : 100
iter = 50


accuracy 0.582385
ctr 0.22105107629248555
precision
0.5823850663939067
NDGC
0.8402040471741848

In [ ]:
accuracy 0.589709
ctr 0.20182517504152558
precision
0.5897094010849513
NDGC
0.8507708285107646